In [16]:
import requests

url = "https://www.baseball-reference.com"  # replace with your URL
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

response = requests.get(url, headers=headers)

if response.status_code == 429:
    retry_after = response.headers.get('Retry-After')
    if retry_after:
        print(f"You have been temporarily banned. Please wait for {retry_after} seconds before retrying.")
    else:
        print("You have been temporarily banned, but the server did not specify a retry time.")
else:
    print("You are not currently banned.")

You are not currently banned.


In [11]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from io import StringIO
import time
import os

years = range(2000, 2024)  # Adjust the range as needed
leagues = ['NL', 'AL']  # National League and American League
statistics_categories = ['standard-batting', 'standard-pitching', 'standard-fielding', 'value-batting', 'value-pitching', 'advanced-batting', 'advanced-pitching', 'win_probability-batting', 'win_probability-pitching']
teams = ['BOS', 'NYY', 'TOR', 'BAL', 'TBR', 'CHW', 'CLE', 'DET', 'KCR', 'MIN', 'HOU', 'LAA', 'OAK', 'SEA', 'TEX', 'ATL', 'MIA', 'NYM', 'PHI', 'WSN', 'CHC', 'CIN', 'MIL', 'PIT', 'STL', 'ARI', 'COL', 'LAD', 'SDP', 'SFG']  # Add all team abbreviations

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

with requests.Session() as s:
    s.headers.update(headers)

    for year in years:
        for league in leagues:
            for category in statistics_categories:
                directory = f"{category}_statistics"
                if not os.path.exists(directory):
                    os.makedirs(directory)

                url = f"https://www.baseball-reference.com/leagues/{league}/{year}-{category}.shtml"
                
                # Construct the file_path directly using the known variables
                file_path = f"{directory}/teams_{category}_{league}_{year}.csv"
                
                if os.path.exists(file_path):
                    print(f"File {file_path} already exists. Skipping request.")
                    continue

                response = s.get(url)
                print(f"Status code for {url}: {response.status_code}")
                
                if response.status_code == 429:
                    retry_after = int(response.headers.get('Retry-After', 10))
                    print(f"Waiting for {retry_after} seconds before retrying...")
                    time.sleep(retry_after)
                    continue

                soup = BeautifulSoup(response.text, 'html.parser')

                tables = soup.find_all('table')
                print(f"Found {len(tables)} tables on {url}")

                for i, table in enumerate(tables):
                    df = pd.read_html(StringIO(str(table)))[0]
                    df.to_csv(file_path, index=False, encoding='utf-8-sig')
                    print(f"Saved data to {file_path}")
                
                time.sleep(2)  # Wait for 1 second before the next request

for team in teams:
    directory = "team_data"
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    for year in years:
        url = f"https://www.baseball-reference.com/teams/{team}/{year}-schedule-scores.shtml"
        
        for i in range(1):  # Assuming there's only one table per page
            table_name = f"team_schedule_{team}_{year}"
            file_path = f"{directory}/{table_name}.csv"
            
            if os.path.exists(file_path):
                print(f"File {file_path} already exists. Skipping request.")
                continue

            response = requests.get(url)
            print(f"Status code for {url}: {response.status_code}")

            if response.status_code == 429:
                retry_after = int(response.headers.get('Retry-After', 10))
                print(f"Waiting for {retry_after} seconds before retrying...")
                time.sleep(retry_after)
                continue

            soup = BeautifulSoup(response.text, 'html.parser')
            table = soup.find('table')
            df = pd.read_html(StringIO(str(table)))[0]
            df.to_csv(file_path, index=False, encoding='utf-8-sig')
            print(f"Saved data to {file_path}")
            
            time.sleep(2)  # Wait for 1 second before the next request only if a request is made

        if not os.path.exists(file_path):
            time.sleep(2)  # Wait for 1 second before the next request only if a request is made

    if not os.path.exists(file_path):
        time.sleep(2)  # Wait for 1 second before the next request only if a request is made

Status code for https://www.baseball-reference.com/leagues/NL/2000-standard-batting.shtml: 200
Found 1 tables on https://www.baseball-reference.com/leagues/NL/2000-standard-batting.shtml
Saved data to standard-batting_statistics/teams_standard-batting_NL_2000.csv
Status code for https://www.baseball-reference.com/leagues/NL/2000-standard-pitching.shtml: 200
Found 1 tables on https://www.baseball-reference.com/leagues/NL/2000-standard-pitching.shtml
Saved data to standard-pitching_statistics/teams_standard-pitching_NL_2000.csv
Status code for https://www.baseball-reference.com/leagues/NL/2000-standard-fielding.shtml: 200
Found 2 tables on https://www.baseball-reference.com/leagues/NL/2000-standard-fielding.shtml
Saved data to standard-fielding_statistics/teams_standard-fielding_NL_2000.csv
Saved data to standard-fielding_statistics/teams_standard-fielding_NL_2000.csv
Status code for https://www.baseball-reference.com/leagues/NL/2000-value-batting.shtml: 200
Found 1 tables on https://www

KeyboardInterrupt: 

In [ ]:
def scrape_tables_from_url(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    with requests.Session() as s:
        s.headers.update(headers)

        response = s.get(url)
        print(f"Status code for {url}: {response.status_code}")
        
        if response.status_code == 429:
            retry_after = int(response.headers.get('Retry-After', 10))
            print(f"Waiting for {retry_after} seconds before retrying...")
            time.sleep(retry_after)
            return

        soup = BeautifulSoup(response.text, 'html.parser')

        tables = soup.find_all('table')
        print(f"Found {len(tables)} tables on {url}")

        for i, table in enumerate(tables):
            df = pd.read_html(StringIO(str(table)))[0]
            file_path = f"table_{i}.csv"
            df.to_csv(file_path, index=False, encoding='utf-8-sig')
            print(f"Saved data to {file_path}")
        
        time.sleep(2)  # Wait for 1 second before the next request

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from io import StringIO
import time
import os

years = range(2000, 2024)  # Adjust the range as needed
leagues = ['NL', 'AL']  # National League and American League
statistics_categories = ['standard-batting', 'standard-pitching', 'standard-fielding', 'value-batting', 'value-pitching', 'advanced-batting', 'advanced-pitching', 'win_probability-batting', 'win_probability-pitching']
teams = ['BOS', 'NYY', 'TOR', 'BAL', 'TBR', 'CHW', 'CLE', 'DET', 'KCR', 'MIN', 'HOU', 'LAA', 'OAK', 'SEA', 'TEX', 'ATL', 'MIA', 'NYM', 'PHI', 'WSN', 'CHC', 'CIN', 'MIL', 'PIT', 'STL', 'ARI', 'COL', 'LAD', 'SDP', 'SFG']  # Add all team abbreviations

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

with requests.Session() as s:
    s.headers.update(headers)

    for year in years:
        for league in leagues:
            for category in statistics_categories:
                directory = f"{category}_statistics"
                if not os.path.exists(directory):
                    os.makedirs(directory)

                url = f"https://www.baseball-reference.com/leagues/{league}/{year}-{category}.shtml"
                
                response = s.get(url)
                print(f"Status code for {url}: {response.status_code}")
                
                if response.status_code == 429:
                    retry_after = int(response.headers.get('Retry-After', 10))
                    print(f"Waiting for {retry_after} seconds before retrying...")
                    time.sleep(retry_after)
                    continue

                soup = BeautifulSoup(response.text, 'html.parser')

                tables = soup.find_all('table')
                print(f"Found {len(tables)} tables on {url}")

                for i, table in enumerate(tables):
                    df = pd.read_html(StringIO(str(table)))[0]
                    # Include the table index in the file path
                    file_path = f"{directory}/teams_{category}_{league}_{year}_table{i}.csv"
                    df.to_csv(file_path, index=False, encoding='utf-8-sig')
                    print(f"Saved data to {file_path}")
                
                time.sleep(2)  # Wait for 1 second before the next request

for team in teams:
    directory = "team_data"
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    for year in years:
        url = f"https://www.baseball-reference.com/teams/{team}/{year}-schedule-scores.shtml"
        
        for i in range(1):  # Assuming there's only one table per page
            table_name = f"team_schedule_{team}_{year}"
            response = requests.get(url)
            print(f"Status code for {url}: {response.status_code}")

            if response.status_code == 429:
                retry_after = int(response.headers.get('Retry-After', 10))
                print(f"Waiting for {retry_after} seconds before retrying...")
                time.sleep(retry_after)
                continue

            soup = BeautifulSoup(response.text, 'html.parser')
            table = soup.find('table')
            df = pd.read_html(StringIO(str(table)))[0]
            # Include the table index in the file path
            file_path = f"{directory}/{table_name}_table{i}.csv"
            df.to_csv(file_path, index=False, encoding='utf-8-sig')
            print(f"Saved data to {file_path}")
            
            time.sleep(2)  # Wait for 1 second before the next request only if a request is made

        if not os.path.exists(file_path):
            time.sleep(2)  # Wait for 1 second before the next request only if a request is made

    if not os.path.exists(file_path):
        time.sleep(2)  # Wait for 1 second before the next request only if a request is made

Status code for https://www.baseball-reference.com/leagues/NL/2000-standard-batting.shtml: 200
Found 1 tables on https://www.baseball-reference.com/leagues/NL/2000-standard-batting.shtml
Saved data to standard-batting_statistics/teams_standard-batting_NL_2000_table0.csv
Status code for https://www.baseball-reference.com/leagues/NL/2000-standard-pitching.shtml: 200
Found 1 tables on https://www.baseball-reference.com/leagues/NL/2000-standard-pitching.shtml
Saved data to standard-pitching_statistics/teams_standard-pitching_NL_2000_table0.csv
Status code for https://www.baseball-reference.com/leagues/NL/2000-standard-fielding.shtml: 200
Found 2 tables on https://www.baseball-reference.com/leagues/NL/2000-standard-fielding.shtml
Saved data to standard-fielding_statistics/teams_standard-fielding_NL_2000_table0.csv
Saved data to standard-fielding_statistics/teams_standard-fielding_NL_2000_table1.csv
Status code for https://www.baseball-reference.com/leagues/NL/2000-value-batting.shtml: 200
F

KeyboardInterrupt: 

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from io import StringIO
import time
import os
import re

years = range(2000, 2024)  # Adjust the range as needed
leagues = ['NL', 'AL']  # National League and American League
statistics_categories = ['standard-batting', 'standard-pitching', 'standard-fielding', 'value-batting', 'value-pitching', 'advanced-batting', 'advanced-pitching', 'win_probability-batting', 'win_probability-pitching']
teams = ['BOS', 'NYY', 'TOR', 'BAL', 'TBR', 'CHW', 'CLE', 'DET', 'KCR', 'MIN', 'HOU', 'LAA', 'OAK', 'SEA', 'TEX', 'ATL', 'MIA', 'NYM', 'PHI', 'WSN', 'CHC', 'CIN', 'MIL', 'PIT', 'STL', 'ARI', 'COL', 'LAD', 'SDP', 'SFG']  # Add all team abbreviations

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

with requests.Session() as s:
    s.headers.update(headers)

    for year in years:
        for league in leagues:
            for category in statistics_categories:
                directory = f"{category}_statistics"
                if not os.path.exists(directory):
                    os.makedirs(directory)

                url = f"https://www.baseball-reference.com/leagues/{league}/{year}-{category}.shtml"
                
                response = s.get(url)
                print(f"Status code for {url}: {response.status_code}")
                
                if response.status_code == 429:
                    retry_after = int(response.headers.get('Retry-After', 10))
                    print(f"Waiting for {retry_after} seconds before retrying...")
                    time.sleep(retry_after)
                    continue

                soup = BeautifulSoup(response.text, 'html.parser')

                tables = soup.find_all('table')
                print(f"Found {len(tables)} tables on {url}")

                for i, table in enumerate(tables):
                    df = pd.read_html(StringIO(str(table)))[0]
                    # Extract the table title
                    table_title = table.find('caption').text if table.find('caption') else f"table{i}"
                    # Replace spaces and special characters in the title with underscores
                    table_title = re.sub(r'\W+', '_', table_title)
                    # Include the table title in the file path
                    file_path = f"{directory}/{table_title}_{category}_{league}_{year}.csv"
                    df.to_csv(file_path, index=False, encoding='utf-8-sig')
                    print(f"Saved data to {file_path}")
                
                time.sleep(2)  # Wait for 1 second before the next request

for team in teams:
    directory = "team_data"
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    for year in years:
        url = f"https://www.baseball-reference.com/teams/{team}/{year}-schedule-scores.shtml"
        
        for i in range(1):  # Assuming there's only one table per page
            table_name = f"team_schedule_{team}_{year}"
            response = requests.get(url)
            print(f"Status code for {url}: {response.status_code}")

            if response.status_code == 429:
                retry_after = int(response.headers.get('Retry-After', 10))
                print(f"Waiting for {retry_after} seconds before retrying...")
                time.sleep(retry_after)
                continue

            soup = BeautifulSoup(response.text, 'html.parser')
            table = soup.find('table')
            df = pd.read_html(StringIO(str(table)))[0]
            # Extract the table title
            table_title = table.find('caption').text if table.find('caption') else f"table{i}"
            # Replace spaces and special characters in the title with underscores
            table_title = re.sub(r'\W+', '_', table_title)
            # Include the table title in the file path
            file_path = f"{directory}/{table_title}_{team}_{year}.csv"
            df.to_csv(file_path, index=False, encoding='utf-8-sig')
            print(f"Saved data to {file_path}")
            
            time.sleep(2)  # Wait for 1 second before the next request only if a request is made

    if not os.path.exists(file_path):
        time.sleep(2)  # Wait for 1 second before the next request only if a request is made

    if not os.path.exists(file_path):
        time.sleep(2)  # Wait for 1 second before the next request only if a request is made

Status code for https://www.baseball-reference.com/leagues/NL/2000-standard-batting.shtml: 200
Found 1 tables on https://www.baseball-reference.com/leagues/NL/2000-standard-batting.shtml
Saved data to standard-batting_statistics/Team_Standard_Batting_Table_standard-batting_NL_2000.csv
Status code for https://www.baseball-reference.com/leagues/NL/2000-standard-pitching.shtml: 200
Found 1 tables on https://www.baseball-reference.com/leagues/NL/2000-standard-pitching.shtml
Saved data to standard-pitching_statistics/Team_Standard_Pitching_Table_standard-pitching_NL_2000.csv
Status code for https://www.baseball-reference.com/leagues/NL/2000-standard-fielding.shtml: 200
Found 2 tables on https://www.baseball-reference.com/leagues/NL/2000-standard-fielding.shtml
Saved data to standard-fielding_statistics/Team_Fielding_Table_standard-fielding_NL_2000.csv
Saved data to standard-fielding_statistics/Player_Standard_Fielding_Table_standard-fielding_NL_2000.csv
Status code for https://www.baseball-

KeyboardInterrupt: 

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from io import StringIO
import time
import os
import re

years = range(2000, 2024)  # Adjust the range as needed
leagues = ['NL', 'AL']  # National League and American League
# statistics_categories = ['standard-batting', 'standard-pitching', 'standard-fielding', 'value-batting', 'value-pitching', 'advanced-batting', 'advanced-pitching', 'win_probability-batting', 'win_probability-pitching']
teams = ['BOS', 'NYY', 'TOR', 'BAL', 'TBR', 'CHW', 'CLE', 'DET', 'KCR', 'MIN', 'HOU', 'LAA', 'OAK', 'SEA', 'TEX', 'ATL', 'MIA', 'NYM', 'PHI', 'WSN', 'CHC', 'CIN', 'MIL', 'PIT', 'STL', 'ARI', 'COL', 'LAD', 'SDP', 'SFG']  # Add all team abbreviations

statistics_categories = ['standard-batting', 'standard-pitching', 'standard-fielding']


headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

with requests.Session() as s:
    s.headers.update(headers)

    for year in years:
        for league in leagues:
            for category in statistics_categories:
                directory = f"{category}_statistics"
                if not os.path.exists(directory):
                    os.makedirs(directory)

                url = f"https://www.baseball-reference.com/leagues/{league}/{year}-{category}.shtml"
                
                # Construct the file_path directly using the known variables
                table_title = f"table_{category}_{league}_{year}"
                file_path = f"{directory}/{table_title}.csv"
                
                if os.path.exists(file_path):
                    print(f"File {file_path} already exists. Skipping request.")
                    continue

                response = s.get(url)
                print(f"Status code for {url}: {response.status_code}")
                
                if response.status_code == 429:
                    retry_after = int(response.headers.get('Retry-After', 10))
                    print(f"Waiting for {retry_after} seconds before retrying...")
                    time.sleep(retry_after)
                    continue

                soup = BeautifulSoup(response.text, 'html.parser')

                tables = soup.find_all('table')
                print(f"Found {len(tables)} tables on {url}")

                for i, table in enumerate(tables):
                    df = pd.read_html(StringIO(str(table)))[0]
                    # Extract the table title
                    table_title = table.find('caption').text if table.find('caption') else f"table{i}"
                    # Replace spaces and special characters in the title with underscores
                    table_title = re.sub(r'\W+', '_', table_title)
                    # Include the table title in the file path
                    file_path = f"{directory}/{table_title}_{category}_{league}_{year}.csv"
                    df.to_csv(file_path, index=False, encoding='utf-8-sig')
                    print(f"Saved data to {file_path}")
                
                time.sleep(2)  # Wait for 1 second before the next request

# for team in teams:
#     directory = "team_data"
#     if not os.path.exists(directory):
#         os.makedirs(directory)
    
#     for year in years:
#         url = f"https://www.baseball-reference.com/teams/{team}/{year}-schedule-scores.shtml"
        
#         # Construct the file_path directly using the known variables
#         table_name = f"team_schedule_{team}_{year}"
#         file_path = f"{directory}/{table_name}.csv"
        
#         if os.path.exists(file_path):
#             print(f"File {file_path} already exists. Skipping request.")
#             continue

#         response = requests.get(url)
#         print(f"Status code for {url}: {response.status_code}")

#         if response.status_code == 429:
#             retry_after = int(response.headers.get('Retry-After', 10))
#             print(f"Waiting for {retry_after} seconds before retrying...")
#             time.sleep(retry_after)
#             continue

#         soup = BeautifulSoup(response.text, 'html.parser')
#         table = soup.find('table')
#         df = pd.read_html(StringIO(str(table)))[0]
#         # Extract the table title
#         table_title = table.find('caption').text if table.find('caption') else f"table{i}"
#         # Replace spaces and special characters in
#         table_title = re.sub(r'\W+', '_', table_title)
#         # Include the table title in the file path
#         file_path = f"{directory}/{table_title}_{team}_{year}.csv"
#         df.to_csv(file_path, index=False, encoding='utf-8-sig')
#         print(f"Saved data to {file_path}")
        
        time.sleep(2)  # Wait for 1 second before the next request

Status code for https://www.baseball-reference.com/leagues/NL/2000-standard-batting.shtml: 200
Found 1 tables on https://www.baseball-reference.com/leagues/NL/2000-standard-batting.shtml
Saved data to standard-batting_statistics/Team_Standard_Batting_Table_standard-batting_NL_2000.csv
Status code for https://www.baseball-reference.com/leagues/NL/2000-standard-pitching.shtml: 200
Found 1 tables on https://www.baseball-reference.com/leagues/NL/2000-standard-pitching.shtml
Saved data to standard-pitching_statistics/Team_Standard_Pitching_Table_standard-pitching_NL_2000.csv
Status code for https://www.baseball-reference.com/leagues/NL/2000-standard-fielding.shtml: 200
Found 2 tables on https://www.baseball-reference.com/leagues/NL/2000-standard-fielding.shtml
Saved data to standard-fielding_statistics/Team_Fielding_Table_standard-fielding_NL_2000.csv
Saved data to standard-fielding_statistics/Player_Standard_Fielding_Table_standard-fielding_NL_2000.csv
Status code for https://www.baseball-

KeyboardInterrupt: 

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO

years = range(2000, 2024)
leagues = ['NL', 'AL']  # Add all leagues here
directory = "player_data"

if not os.path.exists(directory):
    os.makedirs(directory)

for league in leagues:
    for year in years:
        url = f"https://www.baseball-reference.com/leagues/{league}/{year}-standard-pitching.shtml"
        response = requests.get(url)
        divs = soup.find_all('div', class_='table-container')
        for div in divs:
            table = div.find('table')

        for i, table in enumerate(tables):
            df = pd.read_html(StringIO(str(table)))[0]
            # Extract the table title
            table_title = table.find('caption').text if table.find('caption') else f"table{i}"
            # Replace spaces and special characters in
            table_title = re.sub(r'\W+', '_', table_title)
            # Include the table title in the file path
            file_path = f"{directory}/{table_title}_{league}_{year}.csv"
            df.to_csv(file_path, index=False, encoding='utf-8-sig')
            print(f"Saved data to {file_path}")

        time.sleep(2)  # Wait for 1 second before the next request



Saved data to player_data/Team_Standard_Pitching_Table_NL_2000.csv
Saved data to player_data/Team_Standard_Pitching_Table_NL_2001.csv
Saved data to player_data/Team_Standard_Pitching_Table_NL_2002.csv
Saved data to player_data/Team_Standard_Pitching_Table_NL_2003.csv
Saved data to player_data/Team_Standard_Pitching_Table_NL_2004.csv
Saved data to player_data/Team_Standard_Pitching_Table_NL_2005.csv
Saved data to player_data/Team_Standard_Pitching_Table_NL_2006.csv
Saved data to player_data/Team_Standard_Pitching_Table_NL_2007.csv
Saved data to player_data/Team_Standard_Pitching_Table_NL_2008.csv
Saved data to player_data/Team_Standard_Pitching_Table_NL_2009.csv
Saved data to player_data/Team_Standard_Pitching_Table_NL_2010.csv
Saved data to player_data/Team_Standard_Pitching_Table_NL_2011.csv
Saved data to player_data/Team_Standard_Pitching_Table_NL_2012.csv
Saved data to player_data/Team_Standard_Pitching_Table_NL_2013.csv
Saved data to player_data/Team_Standard_Pitching_Table_NL_2014

[<table class="sortable stats_table now_sortable sticky_table eq1 re1 le1" data-cols-to-freeze=",1" id="teams_standard_pitching">
<caption>Team Standard Pitching Table</caption>
<colgroup><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/></colgroup>
<thead>
<tr>
<th aria-label="Tm" class="poptip sort_default_asc left" data-stat="team_name" scope="col">Tm</th>
<th aria-label="#P" class="poptip center" data-stat="pitchers_used" data-tip="&lt;strong&gt;Number of Pitchers used in Games&lt;/strong&gt;" scope="col">#P</th>
<th aria-label="PAge" class="poptip sort_default_asc center" data-stat="age_pitch" data-tip="&lt;strong&gt;Pitchers&amp;#x2019; average age&lt;/strong&gt;&lt;br&gt;Weighted by 3*GS + G + SV" scope="col">PAge</th>
<th aria-label="Runs Allowed/Game" class="poptip center" data-stat="runs_allowed_per_game" data-ti

In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO
import os
import re
import time

# Setup chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Ensure GUI is off
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Set path to chromedriver as per your configuration
webdriver_service = Service(ChromeDriverManager().install())

# Choose Chrome Browser
driver = webdriver.Chrome(service=webdriver_service, options=chrome_options)

leagues = ["NL", "AL"]
years = range(2000, 2024)  # Adjust the range as per your requirement

for league in leagues:
    for year in years:
        url = f"https://www.baseball-reference.com/leagues/{league}/{year}-standard-fielding.shtml"
        driver.get(url)
        time.sleep(5)  # Give the page time to load

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        tables = soup.find_all('table')

        directory = "player_data"
        if not os.path.exists(directory):
            os.makedirs(directory)

        for i, table in enumerate(tables):
            df = pd.read_html(StringIO(str(table)))[0]
            # Extract the table title
            table_title = table.find('caption').text if table.find('caption') else f"table{i}"
            # Replace spaces and special characters in the title with underscores
            table_title = re.sub(r'\W+', '_', table_title)
            # Include the table title in the file path
            file_path = f"{directory}/{league}_{year}_{table_title}.csv"
            df.to_csv(file_path, index=False, encoding='utf-8-sig')
            print(f"Saved data to {file_path}")

driver.quit()


Saved data to player_data/NL_2000_Team_Fielding_Table.csv
Saved data to player_data/NL_2000_Player_Standard_Fielding_Table.csv
Saved data to player_data/NL_2001_Team_Fielding_Table.csv
Saved data to player_data/NL_2001_Player_Standard_Fielding_Table.csv
Saved data to player_data/NL_2002_Team_Fielding_Table.csv
Saved data to player_data/NL_2002_Player_Standard_Fielding_Table.csv
Saved data to player_data/NL_2003_Team_Fielding_Table.csv
Saved data to player_data/NL_2003_Player_Standard_Fielding_Table.csv
Saved data to player_data/NL_2004_Team_Fielding_Table.csv
Saved data to player_data/NL_2004_Player_Standard_Fielding_Table.csv
Saved data to player_data/NL_2004_table2.csv
Saved data to player_data/NL_2005_Team_Fielding_Table.csv
Saved data to player_data/NL_2005_Player_Standard_Fielding_Table.csv
Saved data to player_data/NL_2006_Team_Fielding_Table.csv
Saved data to player_data/NL_2006_Player_Standard_Fielding_Table.csv
Saved data to player_data/NL_2007_Team_Fielding_Table.csv
Saved da

In [21]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO
import os
import re
import time

# Setup chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Ensure GUI is off
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Set path to chromedriver as per your configuration
webdriver_service = Service(ChromeDriverManager().install())

# Choose Chrome Browser
driver = webdriver.Chrome(service=webdriver_service, options=chrome_options)

teams = ['BOS', 'NYY', 'TOR', 'BAL', 'TBR', 'CHW', 'CLE', 'DET', 'KCR', 'MIN', 'HOU', 'LAA', 'OAK', 'SEA', 'TEX', 'ATL', 'MIA', 'NYM', 'PHI', 'WSN', 'CHC', 'CIN', 'MIL', 'PIT', 'STL', 'ARI', 'COL', 'LAD', 'SDP', 'SFG']  # Add all team abbreviations
  # Add all team abbreviations
years = range(2000, 2024)  # Adjust the range as needed
days = range(1, 32)  # Days in a month
months = range(4, 13)  # Months in a year

for team in teams:
    for year in years:
        for month in months:
            for day in days:
                date = f"{year}{str(month).zfill(2)}{str(day).zfill(2)}"
                url = f"https://www.baseball-reference.com/boxes/{team}/{team}{date}0.shtml"
                driver.get(url)
                time.sleep(5)  # Give the page time to load

                # Check if a game exists for the given date
                if "Page Not Found (404 error)" in driver.title:
                    print(f"No game on {date} for {team}. Skipping...")
                    continue

                soup = BeautifulSoup(driver.page_source, 'html.parser')
                tables = soup.find_all('table')

                directory = "boxscore_data"
                if not os.path.exists(directory):
                    os.makedirs(directory)

                for i, table in enumerate(tables):
                    df = pd.read_html(StringIO(str(table)))[0]
                    # Extract the table title
                    table_title = table.find('caption').text if table.find('caption') else f"table{i}"
                    # Replace spaces and special characters in the title with underscores
                    table_title = re.sub(r'\W+', '_', table_title)
                    # Include the table title in the file path
                    file_path = f"{directory}/{team}_{date}_{table_title}.csv"
                    df.to_csv(file_path, index=False, encoding='utf-8-sig')
                    print(f"Saved data to {file_path}")

driver.quit()


No game on 20000401 for BOS. Skipping...
No game on 20000402 for BOS. Skipping...
No game on 20000403 for BOS. Skipping...
No game on 20000404 for BOS. Skipping...
No game on 20000405 for BOS. Skipping...
No game on 20000406 for BOS. Skipping...
No game on 20000407 for BOS. Skipping...
No game on 20000408 for BOS. Skipping...
No game on 20000409 for BOS. Skipping...
No game on 20000410 for BOS. Skipping...
Saved data to boxscore_data/BOS_20000411_table0.csv
Saved data to boxscore_data/BOS_20000411_table1.csv
Saved data to boxscore_data/BOS_20000411_table2.csv
Saved data to boxscore_data/BOS_20000411_table3.csv
Saved data to boxscore_data/BOS_20000411_table4.csv
Saved data to boxscore_data/BOS_20000411_table5.csv
Saved data to boxscore_data/BOS_20000411_table6.csv
Saved data to boxscore_data/BOS_20000411_table7.csv
Saved data to boxscore_data/BOS_20000411_table8.csv
Saved data to boxscore_data/BOS_20000411_table9.csv
Saved data to boxscore_data/BOS_20000411_table10.csv
Saved data to box

KeyboardInterrupt: 

In [75]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO
import os
import re
import time

# Setup chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Ensure GUI is off
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Set path to chromedriver as per your configuration
webdriver_service = Service(ChromeDriverManager().install())

# Choose Chrome Browser
driver = webdriver.Chrome(service=webdriver_service, options=chrome_options)

teams = ['BOS', 'NYY', 'TOR', 'BAL', 'TBR', 'CHW', 'CLE', 'DET', 'KCR', 'MIN', 'HOU', 'LAA', 'OAK', 'SEA', 'TEX', 'ATL', 'MIA', 'NYM', 'PHI', 'WSN', 'CHC', 'CIN', 'MIL', 'PIT', 'STL', 'ARI', 'COL', 'LAD', 'SDP', 'SFG']
years = range(2000, 2024)
days = range(1, 32)
months = range(4, 13)

for team in teams:
    for year in years:
        for month in months:
            for day in days:
                date = f"{year}{str(month).zfill(2)}{str(day).zfill(2)}"
                url = f"https://www.baseball-reference.com/boxes/{team}/{team}{date}0.shtml"
                driver.get(url)
                time.sleep(5)

                if "Page Not Found (404 error)" in driver.title:
                    print(f"No game on {date} for {team}. Skipping...")
                    continue

                soup = BeautifulSoup(driver.page_source, 'html.parser')
                tables = soup.find_all('table')

                directory = "boxscore_data"
                if not os.path.exists(directory):
                    os.makedirs(directory)

                table_names = set()  # Keep track of table names
                for i, table in enumerate(tables):
                    df = pd.read_html(StringIO(str(table)))[0]
                    table_title = table.find('caption').text if table.find('caption') else f"table{i}"
                    table_title = re.sub(r'\W+', '_', table_title)



                    # Check if table name is duplicate
                    if table_title in table_names:
                        table_title += "_pitchers"
                    table_names.add(table_title)

                    file_path = f"{directory}/{team}_{date}_{table_title}.csv"
                    df.to_csv(file_path, index=False, encoding='utf-8-sig')
                    print(f"Saved data to {file_path}")
                    
    

driver.quit()

No game on 20000401 for BOS. Skipping...
No game on 20000402 for BOS. Skipping...
No game on 20000403 for BOS. Skipping...
No game on 20000404 for BOS. Skipping...
No game on 20000405 for BOS. Skipping...
No game on 20000406 for BOS. Skipping...
No game on 20000407 for BOS. Skipping...
No game on 20000408 for BOS. Skipping...
No game on 20000409 for BOS. Skipping...
No game on 20000410 for BOS. Skipping...
Saved data to boxscore_data/BOS_20000411_table0.csv
Saved data to boxscore_data/BOS_20000411_Starting_Lineup.csv
Saved data to boxscore_data/BOS_20000411_table1.csv
Saved data to boxscore_data/BOS_20000411_Starting_Lineup.csv
Saved data to boxscore_data/BOS_20000411_table2.csv
Saved data to boxscore_data/BOS_20000411_Starting_Lineup.csv
Saved data to boxscore_data/BOS_20000411_table3.csv
Saved data to boxscore_data/BOS_20000411_Starting_Lineup.csv
Saved data to boxscore_data/BOS_20000411_table4.csv
Saved data to boxscore_data/BOS_20000411_Starting_Lineup.csv
Saved data to boxscore_da

KeyboardInterrupt: 

In [38]:
print(tables.index('Boston Red Sox'))


ValueError: 'Boston Red Sox' is not in list

In [76]:
# print(tables)
tables = soup.find_all('table')

captions = []
lineup = pd.DataFrame()  # Initialize lineup as an empty DataFrame
for table in tables:
    caption = table.find('caption')
    rows = table.find_all('tr')  # Find all rows in the table
    if caption and len(rows) == 10:  # Check if there are 10 rows
        captions.append(caption)
        df = pd.read_html(StringIO(str(table)))[0]  # Convert table to DataFrame
        lineup = pd.concat([lineup, df], axis=1)  # Concatenate the DataFrame to lineup


In [79]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO
import os
import re
import time

# Setup chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Ensure GUI is off
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Set path to chromedriver as per your configuration
webdriver_service = Service(ChromeDriverManager().install())

# Choose Chrome Browser
driver = webdriver.Chrome(service=webdriver_service, options=chrome_options)

teams = ['BOS', 'NYY', 'TOR', 'BAL', 'TBR', 'CHW', 'CLE', 'DET', 'KCR', 'MIN', 'HOU', 'LAA', 'OAK', 'SEA', 'TEX', 'ATL', 'MIA', 'NYM', 'PHI', 'WSN', 'CHC', 'CIN', 'MIL', 'PIT', 'STL', 'ARI', 'COL', 'LAD', 'SDP', 'SFG']
years = range(2000, 2024)
days = range(1, 32)
months = range(4, 13)

for team in teams:
    for year in years:
        for month in months:
            for day in days:
                date = f"{year}{str(month).zfill(2)}{str(day).zfill(2)}"
                url = f"https://www.baseball-reference.com/boxes/{team}/{team}{date}0.shtml"
                driver.get(url)
                time.sleep(5)

                if "Page Not Found (404 error)" in driver.title:
                    print(f"No game on {date} for {team}. Skipping...")
                    continue

                soup = BeautifulSoup(driver.page_source, 'html.parser')
                tables = soup.find_all('table')

                directory = "boxscore_data"
                if not os.path.exists(directory):
                    os.makedirs(directory)

                table_names = set()  # Keep track of table names
                lineup = pd.DataFrame()  # Initialize lineup as an empty DataFrame
                for i, table in enumerate(tables):
                    df = pd.read_html(StringIO(str(table)))[0]
                    table_title = table.find('caption').text if table.find('caption') else f"table{i}"
                    table_title = re.sub(r'\W+', '_', table_title)

                    # Check if table name is duplicate
                    if table_title in table_names:
                        table_title += "_pitchers"
                    table_names.add(table_title)

                    file_path = f"{directory}/{team}_{date}_{table_title}.csv"
                    df.to_csv(file_path, index=False, encoding='utf-8-sig')
                    print(f"Saved data to {file_path}")

                    # Check if there are 10 rows
                    if len(table.find_all('tr')) == 10:
                        # Convert table to DataFrame
                        lineup_df = pd.read_html(StringIO(str(table)))[0]
                        # Concatenate the DataFrame to lineup
                        lineup = pd.concat([lineup, lineup_df], axis=1)

                # Save the lineup DataFrame to a CSV file
                lineup_file_path = f"{directory}/{team}_{date}_lineup.csv"
                lineup.to_csv(lineup_file_path, index=False, encoding='utf-8-sig')
                print(f"Saved lineup data to {lineup_file_path}")

driver.quit()

No game on 20000401 for BOS. Skipping...
No game on 20000402 for BOS. Skipping...
No game on 20000403 for BOS. Skipping...
No game on 20000404 for BOS. Skipping...
No game on 20000405 for BOS. Skipping...
No game on 20000406 for BOS. Skipping...
No game on 20000407 for BOS. Skipping...
No game on 20000408 for BOS. Skipping...
No game on 20000409 for BOS. Skipping...
No game on 20000410 for BOS. Skipping...
Saved data to boxscore_data/BOS_20000411_table0.csv
Saved data to boxscore_data/BOS_20000411_table1.csv
Saved data to boxscore_data/BOS_20000411_table2.csv
Saved data to boxscore_data/BOS_20000411_table3.csv
Saved data to boxscore_data/BOS_20000411_table4.csv
Saved data to boxscore_data/BOS_20000411_table5.csv
Saved data to boxscore_data/BOS_20000411_table6.csv
Saved data to boxscore_data/BOS_20000411_table7.csv
Saved data to boxscore_data/BOS_20000411_table8.csv
Saved data to boxscore_data/BOS_20000411_table9.csv
Saved data to boxscore_data/BOS_20000411_table10.csv
Saved data to box

TimeoutException: Message: timeout: Timed out receiving message from renderer: 300.000
  (Session info: chrome-headless-shell=120.0.6099.110)
Stacktrace:
	GetHandleVerifier [0x00826EE3+174339]
	(No symbol) [0x00750A51]
	(No symbol) [0x00466FF6]
	(No symbol) [0x00459B86]
	(No symbol) [0x00459ABF]
	(No symbol) [0x004589DC]
	(No symbol) [0x00458EFD]
	(No symbol) [0x004622F5]
	(No symbol) [0x0046F4F4]
	(No symbol) [0x00472676]
	(No symbol) [0x004593BB]
	(No symbol) [0x0046F41C]
	(No symbol) [0x004CA05B]
	(No symbol) [0x004B6DA6]
	(No symbol) [0x00491034]
	(No symbol) [0x00491F8D]
	GetHandleVerifier [0x008C4B1C+820540]
	sqlite3_dbdata_init [0x009853EE+653550]
	sqlite3_dbdata_init [0x00984E09+652041]
	sqlite3_dbdata_init [0x009797CC+605388]
	sqlite3_dbdata_init [0x00985D9B+656027]
	(No symbol) [0x0075FE6C]
	(No symbol) [0x007583B8]
	(No symbol) [0x007584DD]
	(No symbol) [0x00745818]
	BaseThreadInitThunk [0x75A57BA9+25]
	RtlInitializeExceptionChain [0x775ABD2B+107]
	RtlClearBits [0x775ABCAF+1

In [77]:
len(caption.text.split())
print(lineup)

     0                 1   2    0                  1   2
0  1.0  Ryan Christenson  CF  1.0       Darren Lewis  LF
1  2.0     Miguel Tejada  SS  2.0         Trot Nixon  RF
2  3.0      Olmedo Saenz  1B  3.0      Brian Daubach  DH
3  4.0         John Jaha  DH  4.0  Nomar Garciaparra  SS
4  5.0     Jeremy Giambi  RF  5.0       Carl Everett  CF
5  6.0   Ramon Hernandez   C  6.0       Mike Stanley  1B
6  7.0        Ben Grieve  LF  7.0          Jeff Frye  2B
7  8.0    Jorge Velandia  2B  8.0      Jason Varitek   C
8  9.0       Eric Chavez  3B  9.0       Wilton Veras  3B
9  NaN       Gil Heredia   P  NaN       Jeff Fassero   P
